In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy
import requests
import json
import re
import csv

In [24]:
addresses = pd.read_csv('data/addresses/aave_addresses.csv')

temp1 = pd.read_csv('data/exchanges/temp1.csv')
temp2 = pd.read_csv('data/exchanges/temp2.csv')

CEX_addresses = pd.read_csv('data/exchanges/cex.csv')
CEX = set(CEX_addresses['address'].str.lower())

EOA = set(addresses.loc[addresses['type'] == False, 'address'])
CA = set(addresses.loc[addresses['type'] == True, 'address'])

temp1 = set(temp1['address'].str.lower())
temp2 = set(temp2['address'].str.lower())

In [ ]:
filtered_temp2 = temp2[~temp2['address'].str.lower().isin(CEX)]
filtered_temp2['address'] = filtered_temp2['address'].str.lower()
filtered_temp2[['address', 'Label']].to_csv('filtered_temp2.csv', index=False)

C:\Users\Sam\AppData\Local\Temp\ipykernel_18660\1363869768.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_temp2['address'] = filtered_temp2['address'].str.lower()


In [ ]:
with open('data/exchanges/cex_evms_addresses.sql', 'r', encoding='utf-8') as f:
    content = f.read()

rows = re.findall(r'\((0x[a-fA-F0-9]+),\s*\'([^\']+)\'', content)

with open('data/exchanges/cex.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['address', 'cex_name'])
    writer.writerows(rows)

In [5]:
addresses = pd.read_csv('data/addresses/combined_addresses.csv')

In [7]:
filtered_addresses = addresses[addresses['type'] == False]
filtered_addresses[['address']].to_csv('data/addresses/EOA_addresses.csv', index=False)

In [3]:
address_info = {}
with open('data/exchanges/exchanges.csv', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        address = row['address'].lower()
        address_info[address] = {
            'label1': row.get('label', '')
        }

with open('data/exchanges/exchanges2.csv', newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        address = row['address'].lower()
        if address not in address_info:
            address_info[address] = {}
        address_info[address].update({
            'type': row.get('type', ''),
            'name': row.get('name', '')
        })
        
with open('data/exchanges/exchanges3.csv', newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        address = row['address'].lower()
        if address not in address_info:
            address_info[address] = {}
        address_info[address].update({
            'label2': row.get('label', '')
        })

wallet_exchange_map = {}
with open('data/exchanges/exchanges.js') as f:
    content = f.read()

matches = re.findall(r'(?:["\']?)([a-zA-Z0-9_-]+)(?:["\']?)\s*:\s*\[\s*([^\]]+?)\s*\]', content, re.DOTALL)

for exchange, addrs_raw in matches:
    addresses = re.findall(r"'(0x[a-fA-F0-9]+)'", addrs_raw)
    for addr in addresses:
        wallet_exchange_map[addr.lower()] = exchange

for address, exchange in wallet_exchange_map.items():
    if address not in address_info:
        address_info[address] = {}
    address_info[address]['exchange'] = exchange
    
with open('data/exchanges/exchanges.json', encoding='utf-8') as f:
    json_data = json.load(f)

for address, exchange in json_data.items():
    address = address.lower()
    if address not in address_info:
        address_info[address] = {}
    address_info[address]['exchange'] = exchange

with open('data/exchanges/all_exchanges.csv', 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['address', 'label1', 'label2', 'type', 'name', 'exchange']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for address, data in address_info.items():
        row = {
            'address': address,
            'label1': data.get('label1', ''),
            'label2': data.get('label2', ''),
            'type': data.get('type', ''),
            'name': data.get('name', ''),
            'exchange': data.get('exchange', '')
        }
        writer.writerow(row)

In [ ]:
token_decimals = {
    'aArbWETH': 10**18,
    'cArbWETH': 10**18,
    'aArbUSDC': 10**6,
    'cArbUSDC': 10**6,
    'aArbUSDT': 10**6,
    'cArbUSDT': 10**6,
    'aArbAAVE': 10**18,
    'ArbAAVE': 10**18,
    'ArbCOMP': 10**18
}

comp_tokens = ['COMP', 'cCOMP', 'cWETHv3', 'cUSDTv3', 'cUSDCv3']
aave_tokens = ['AAVE', 'stkAAVE', 'aEthWETH', 'aEthUSDT', 'aEthUSDC']
opt_tokens = ['aOptWETH', 'aOptUSDC', 'aOptUSDT', 'OptAAVE', 'cOptWETH', 'cOptUSDC', 'cOptUSDT', 'OptCOMP']
pol_tokens = ['aPolWETH', 'aPolUSDC', 'aPolUSDT', 'PolAAVE', 'cPolUSDT', 'PolCOMP']
bas_tokens = ['aBasWETH', 'aBasUSDC', 'cBasWETH', 'cBasUSDC', 'BasCOMP']
arb_tokens = ['aArbWETH', 'aArbUSDC', 'aArbUSDT', 'ArbAAVE', 'cArbWETH', 'cArbUSDC', 'cArbUSDT', 'ArbCOMP']

def scale_transfer_value(token, market):
    df = pd.read_csv(f'data/{market}/{token}.csv')
    df['value'] = df['value'].astype(float) / token_decimals[token]
    df.to_csv(f'data/{market}/{token}.csv', index=False)
    
for token in arb_tokens:
    scale_transfer_value(token, 'arbitrum')

In [7]:
df_list = []

for token in aave_tokens:
    df = pd.read_csv(f'data/aave/{token}.csv')
    df['token'] = token
    
    df_list.append(df)
    
for token in comp_tokens:
    df = pd.read_csv(f'data/compound/{token}.csv')
    df['token'] = token
    
    df_list.append(df)

for token in opt_tokens:
    df = pd.read_csv(f'data/optimism/{token}.csv')
    df['token'] = token
    
    df_list.append(df)
    
for token in pol_tokens:
    df = pd.read_csv(f'data/polygon/{token}.csv')
    df['token'] = token
    
    df_list.append(df)
    
for token in bas_tokens:
    df = pd.read_csv(f'data/base/{token}.csv')
    df['token'] = token
    
    df_list.append(df)
    
for token in arb_tokens:
    df = pd.read_csv(f'data/arbitrum/{token}.csv')
    df['token'] = token
    
    df_list.append(df)
    
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv('data/complete/combined_transfers.csv', index=False)

In [ ]:
with open('data/proposals/compound_proposals.json', 'r', encoding="utf8") as f:
    proposals_data = json.load(f)
    
with open('data/proposals/compound_votes.json', 'r', encoding="utf8") as f:
    votes_data = json.load(f)

In [ ]:
def scale_proposals(proposals_data):
    for proposal in proposals_data['data']['proposals']:
        proposal['abstainWeightedVotes'] = str(int(proposal['abstainWeightedVotes']) / 10**18)
        proposal['againstWeightedVotes'] = str(int(proposal['againstWeightedVotes']) / 10**18)
        proposal['forWeightedVotes'] = str(int(proposal['forWeightedVotes']) / 10**18)
        proposal['quorumVotes'] = str(int(proposal['quorumVotes']) / 10**18)
        proposal['totalWeightedVotes'] = str(int(proposal['totalWeightedVotes']) / 10**18)
    return proposals_data

scaled_proposals = scale_proposals(proposals_data)

with open('data/proposals/compound_scaled_proposals.json', 'w') as f:
    json.dump(scaled_proposals, f, indent=4)

In [ ]:
def scale_votes(votes_data):
    for proposal in votes_data:
        for vote in proposal['votes']:
            vote['weight'] = str(int(vote['weight']) / 10**18)
    return votes_data

scaled_votes = scale_votes(votes_data)

with open('data/proposals/compound_scaled_votes.json', 'w') as f:
    json.dump(scaled_votes, f, indent=4)

In [7]:
with open('data/proposals/aave/aave_v2_on_chain_proposals_1_year.json', 'r', encoding="utf8") as f:
    aave_v2_proposals = json.load(f)

with open('data/proposals/aave/aave_v3_on_chain_proposals_1_year.json', 'r', encoding="utf8") as f:
    aave_v3_proposals = json.load(f)
    
with open('data/proposals/aave/aave_v2_on_chain_votes_1_year.json', 'r', encoding="utf8") as f:
    aave_v2_votes = json.load(f)

with open('data/proposals/aave/aave_v3_on_chain_votes_1_year.json', 'r', encoding="utf8") as f:
    aave_v3_votes = json.load(f)

In [14]:
for proposal in aave_v2_proposals['data']['proposals']:
    proposal['currentNoVote'] = str(int(proposal['currentNoVote']) / 10**18)
    proposal['currentYesVote'] = str(int(proposal['currentYesVote']) / 10**18)
    
with open('data/proposals/aave/aave_v2_on_chain_proposals_1_year.json', 'w') as f:
    json.dump(aave_v2_proposals, f, indent=4)

In [13]:
for proposal in aave_v3_proposals['data']['proposals']:
    transactions = proposal['transactions']
    if transactions.get('executed'):
        proposal['state'] = "Executed"
    elif transactions.get('canceled'):
        proposal['state'] = "Canceled"
    elif transactions.get('failed'):
        proposal['state'] = "Failed"
    else:
        proposal['state'] = "Active"
        
    votes = proposal.get('votes')
    if votes:
        votes['againstVotes'] = str(int(votes['againstVotes']) / 10**18)
        votes['forVotes'] = str(int(votes['forVotes']) / 10**18)
        
    for voters in aave_v3_votes:
        if voters['proposal_id'] == int(proposal['id']):
            proposal['totalCurrentVoters'] = len(voters['votes'])
    
with open('data/proposals/aave/aave_v3_on_chain_proposals_1_year.json', 'w') as f:
    json.dump(aave_v3_proposals, f, indent=4)

In [3]:
for votes in aave_v2_votes:
    for vote in votes['votes']:
        vote['weight'] = str(int(vote['weight']) / 10**18)
        
with open('data/proposals/aave/aave_v2_on_chain_votes_1_year.json', 'w') as f:
    json.dump(aave_v2_votes, f, indent=4)

In [5]:
for votes in aave_v3_votes:
    for vote in votes['votes']:
        if vote.get('choice') == 0:
            vote['choice'] = "AGAINST"
        elif vote.get('choice') == 1:
            vote['choice'] = "FOR"
        
        vote['weight'] = str(vote['weight'] / 10**18)
        
with open('data/proposals/aave/aave_v3_on_chain_votes_1_year.json', 'w') as f:
    json.dump(aave_v3_votes, f, indent=4)

In [2]:
with open('data/proposals/compound/compound_votes_1_year.json', 'r', encoding="utf8") as f:
    comp_votes = json.load(f)

In [3]:
for votes in comp_votes:
    for vote in votes['votes']:
        vote['weight'] = str(int(vote['weight']) / 10**18)
        
with open('data/proposals/compound/compound_votes_1_year.json', 'w') as f:
    json.dump(comp_votes, f, indent=4)